<html><b>Libraries to include all the required packages. 
twitteR for using the twitter functions such as "searchTwitter", "twListToDF".
ggmap for using geocode() function.
maps and ggplot to plot the map.</b></html>

In [ ]:
library("twitteR")
library("ggmap")
library("maps")
library("maptools")
library("ggplot2")

<html><b>Twitter Credentials to programatically work with twitter.</b></html>

In [ ]:
consumer_key = "u0dJDMOt3wZJsS5HSp9mU0Un1"
consumer_secret = "FUTqk8V9ROhzXds1uXdIYC40S6058Ocu5fTzNh97HWPnuk0X54"
access_token = "71994973-7usKrMfFnM6BAMp1jaLyWQ6Jo0pFbo6FbigNSVsN9"
access_secret = "Xp3DdDJa2j1nvQxML7hppvti6j1RiEnrp802UwjBox6oU"
setup_twitter_oauth(consumer_key, consumer_secret, access_token, access_secret)

<html><b>Query to be searched.</b></html>

In [ ]:
query <- "Super Bowl"

<html><b>Collect Tweets using searchTwitter</b></html>

In [ ]:
results <- searchTwitter(query, n = 30000)  # Gather Tweets 

<html><b>Convert the collected tweets into data frame.</b></html>

In [ ]:
tweetFrame <- twListToDF(results)  

<html><b>Write the tweets into a file called "tweets.csv". </b></html>

In [ ]:
write.csv(tweetFrame, file="tweets.csv")

<html><b>Read the tweets into a file called "tweets.csv". </b></html>

In [ ]:
userInfo <- read.csv(file="tweets.csv")

<html><b>Lookup for the screen name and convert the information into a data frame.</b></html>

In [ ]:
userInfo <- lookupUsers(userInfo$screenName)  # Batch lookup of user info
userFrame <- twListToDF(userInfo) 

<html><b>Convert the collected user information into sets of 2500 to find out geocode.</b></html>

In [ ]:
set1 <- userFrame[1:2500,]
set2 <- userFrame[2501:5000,]
set3 <- userFrame[5001:6000,]
set4 <- userFrame[7501:10000,]

<html><b>Retaining only those user information which has non-null location.</b></html>

In [ ]:
locatedUsers1 <- !is.na(set1$location)  # Keep only users with location info
locatedUsers2 <- !is.na(set2$location)
locatedUsers3 <- !is.na(set3$location)
locatedUsers4 <- !is.na(set4$location)

<html><b>Collecting the geocode of the locations.</b></html>

In [ ]:
locations1 <- geocode(set1$location[locatedUsers1])  # Use amazing API to guess
locationName1 <- set1$location[locatedUsers1]

In [ ]:
locations2 <- geocode(set2$location[locatedUsers2])  # Use amazing API to guess
locationName2 <- set2$location[locatedUsers2]

<html><b>Retaining only those locations for set1 which have valid latitude and longitude locations.</b></html>

In [ ]:
tweetLocation1 <- data.frame()
tmp <- data.frame()
for(i in 1:length(locations1$lat)){    
    if(!(is.na(locations1$lat[i])) || !(is.na(locations1$lon[i])))
   {
        data_frame <- data.frame(locationName=locationName1[i],lat=locations1$lat[i],long=locations1$lon[i])
        tweetLocation1 <- rbind(tweetLocation1, data_frame)
    }
}

<html><b>Retaining only those locations for set2 which have valid latitude and longitude locations.</b></html>

In [ ]:
tweetLocation2 <- data.frame()
tmp <- data.frame()
for(i in 1:length(locations2$lat)){
    
    
    if(!(is.na(locations2$lat[i])) || !(is.na(locations2$lon[i])))
   {       
        data_frame <- data.frame(locationName=locationName2[i],lat=locations2$lat[i],long=locations2$lon[i])
        tweetLocation2 <- rbind(tweetLocation2, data_frame)
    }
}


<html><b>Code to merge all the individual data frames of size 2500 here.</b></html>

In [ ]:
merge1 =rbind(tweetLocation1,tweetLocation2)

<html><b>Write the geocodes of the locations in a file.</b><html>

In [ ]:
write.csv(merge1, file="DIC_PA1.csv")

<html><b>Read the geocodes of the locations from the file.</b><html>

In [ ]:
df <- read.csv(file= "DIC_PA1.csv")      # in the followung code replace tweetFrame with df

<html><b>Find the frequency of the tweets by aggregating them according to their latitude and longitude.</b><html>

In [ ]:
plotMap  <- aggregate(rep(1, length(paste0(df$lat, df$long))),
                             by=list(df$lat, df$long), sum)

<html><b>Keeping only the locations of USA. </b></html>

In [ ]:
finalFrame= data.frame()
for(i in 1:length(plotMap$Group.1))
    {
    if(plotMap$Group.2[i] > -140 && plotMap$Group.2[i] < -60 && plotMap$Group.1[i] >20 && plotMap$Group.1[i] < 60){
        data_frame_temp = data.frame(lat=plotMap$Group.1[i], long=plotMap$Group.2[i], x=plotMap$x[i])
        finalFrame=rbind(finalFrame, data_frame_temp)
    }    
}

<html><b>Write the geocodes and their frequency in a file.</b></html>

In [ ]:
write.csv(finalFrame, file="DIC_PA1_Plot3.csv")

<html><b>Reading the above created file. </b></html>

In [ ]:
df <- read.csv(file= "DIC_PA1_Plot3.csv")
df

<html><b>Plotting the points on the USA map. The size of points is changed using the "size" parameter in the geom_point() function. </b></html>

In [ ]:
usa_center = as.numeric(geocode("United States"))
USAMap = ggmap(get_googlemap(center=usa_center,scale=2, zoom=4), extent= "normal") 
USAMap +  
    geom_point(aes(x=long, y=lat), data=df, col="red", alpha=0.4, size=(df$x/5) )